# Design a youtube like app

# Solution 

## Data Models

#### User 
The user data model can be stored in a single SQL like DB, since we're unlinkely to see more than a few 10s of Billions of users (if every person in the world had 5 accounts, we'd have 35 billion accounts).  There are also companies, and organizations, but those probably are much smaller in number than people type users.  Should we distinguish between user types?  That's a more technical/application use level question.

A user should have

(user_id, name, email, login info, registration date, personal data, other meta-data)

We can also break this into two tables based on use, one for login data and another for user profile data

(user_id, name, email, login_info, registration date)

and

(user_id, name, personal data, other meta-data)


The second major object is the content, video and images.  Youtube has both of these, the video and stills of the videos for various uses.  Videos contain a lot of meta-data

#### Video info

(video_id, author_id, upload_date, title, description, interaction data (views, likes, shares, ...), category information, length, location, ..., first_thumbnail_id)

Notice that this table doesn't actually contain the video file, but it does contain a pointer to the video_id.  The videos themselves  should be stored in a way that optimizes playback.  That will likely include

- the video disected into chunks for piece by peice ingestion
- several copies of the entire video pieces at different resolutions

We probably end up with a schema that looks like

(video_id, timestamp_from_start, thumbnail_id, video_data)

This would best be stored in DB that is a big KV store that supports secondary indexing by the timestamp_from_start value.

#### Images

To support the UX, we need to have thumbnails of the images to show.  These would require less space, but would look the same as above.

(thumbnail_id, video_id, 


#### Author <-> video

An author video relation should be stored seperately.  This allows us to keep a full index of the videos owned by a particular user.

(user_id, video_id_1, video_id_2, ...)

This brings ups a question about access.  This order matters.  If we have to re-shuffle them in server before showing, we coudl be waisiting time.  When we write and update these rows, we should consider things like popularity, upload date, pinning (author prioritizing the video) and so on.

We may even go as far as to create a truncated DB that creates the "first page results" for every user and their videos.  This table would be similar to the one above, but would be bounded in it's row length.

If we commit the the idea of a first page result, we can actually then break down the (author, video) table into a kv store that looks more like this.  This starts to look like the video info table, and in-fact may be redundant, but is indexed by author instead of video_id.

#### User <-> video liked table (basic interaction vs detailed interaction table)

This table is to keep track of videos users have watched.  This is used mostly for analytics and recommendations.  It should be broken up into two or more versions.  The first being a very basic table for fast access.

(user, video_id_1, video_id_2, ...)

This talbe keeps track of all the videos watched, for any length of time.  This allows for basic user item recommendation, user user similarity and item-item similarity.  We simplify the interactions to binary watch/hasn't watched and can find video and user neighbors.

Then the most detailed version of intereactions should be kept in logs.

The logs will be post-processed and summerized as needed.  There would likely be lots of different versions of log summerizations.

#### CDN, and caching

We would need to think carefully about how to prioritize caching and storage of videos based on a few factors.

- recency of upload
- recency of view
- frequency of view
- trending (combination of timeXfrequencyXrecency)
- region
- language
- category (durign the olympics, we should be making older olympics videos more highly available)

We need a high-availability layer that caches videos per region, and some kind of deep storage for the long tail.  Geo-replication is still necesarry even for the long-tail of videos.
